In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
#loading dataset
df = pd.read_csv('Data/ship_fuel_efficiency.csv')

In [4]:
df.columns

Index(['ship_id', 'ship_type', 'route_id', 'month', 'distance', 'fuel_type',
       'fuel_consumption', 'CO2_emissions', 'weather_conditions',
       'engine_efficiency', 'fuel_efficiency', 'fuel_usage'],
      dtype='object')

In [7]:
X = df.drop('fuel_usage', axis=1)
y = df['fuel_usage']

In [8]:
#Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
#Scaling data for model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['float64']))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=['float64']))

In [13]:
#model selection
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

Forgot to encode categorical data. Need to do this step and re-attempt

In [14]:
#Evaluation
y_pred = model.predict(X_test_scaled)
print("Classification Report:")
print(classification_report(y_test, y_pred))

TypeError: Cannot convert numpy.ndarray to numpy.ndarray